# Binance API

In [ ]:
# Setup
from types import MethodDescriptorType
import requests
import hashlib
import threading
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

## Collect Data

In [ ]:
## Set parameters
root_url = 'https://api.binance.com/api/v1/klines'
symbol = 'NMRUSDT' # 'ETHUSDT'
interval = '1d'
url = root_url + '?symbol=' + symbol + '&interval=' + interval
print(url)

In [ ]:
def get_bars(symbol, interval):
    """
    Collect data from Binance API
    """
    url = root_url + '?symbol=' + symbol + '&interval=' + interval
    data = dict()
    data['limit'] = 1_000 # 290
    data = json.loads(requests.get(url, params=data).text)
    df = pd.DataFrame(data)
    df.columns = ['open_time',
                  'open', 'high', 'low', 'close', 'volume',
                  'close_time', 'qav', 'num_trades',
                  'taker_base_vol', 'taker_quote_vol', 'ignore']
    df.index = [dt.datetime.fromtimestamp(x/1000.0) for x in df.close_time]
    return df

In [ ]:
## // Collect Data //
df = get_bars(symbol,interval)

In [ ]:
df.shape

In [ ]:
# Check dataset
df.head()

In [ ]:
# Fix Column types
df[['open', 'high', 'low', 'close', 'volume']] = df[['open', 'high', 'low', 'close', 'volume']].apply(pd.to_numeric)

df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
df['close_time'] = pd.to_datetime(df['close_time'], unit='ms')

In [ ]:
df.dtypes

In [ ]:
print("First date:", df["open_time"].min())
print("Last date:", df["open_time"].max())

## Create Features

In [ ]:
features = ['open_time', 'close_time',
            'open','high','low','close','volume','num_trades']

df = df[features].copy(deep=True)

### Rolling Mean-Std 

In [11]:
# Calculate rolling mean/std
window = 10

df[f'MA_{window}'] = df['close'].rolling(window).mean()
df[f'STD_{window}'] = df['close'].rolling(window).std()

In [12]:
# Check dataset
df.head()

,open_time,close_time,open,high,low,close,volume,num_trades,MA_10,STD_10
2020-08-20 02:59:59.999,2020-08-19,2020-08-19 23:59:59.999,50.420,50.420,37.770,42.079,11838.172,4880,NaN,NaN
2020-08-21 02:59:59.999,2020-08-20,2020-08-20 23:59:59.999,42.281,53.880,39.542,46.930,62972.514,18560,NaN,NaN
2020-08-22 02:59:59.999,2020-08-21,2020-08-21 23:59:59.999,46.952,52.100,41.352,41.896,47528.204,19241,NaN,NaN
2020-08-23 02:59:59.999,2020-08-22,2020-08-22 23:59:59.999,42.181,51.000,41.072,47.978,19932.147,8238,NaN,NaN
2020-08-24 02:59:59.999,2020-08-23,2020-08-23 23:59:59.999,48.048,48.057,42.828,43.591,17284.855,6039,NaN,NaN


### Price changes

In [13]:
df['Price_Change'] = np.round(df['close'] - df['close'].shift(1),2)

In [14]:
# Seeing how much price changed in consecutive days (in percentage)
df['Price_Change%'] = np.round( ((df['close'] - df['close'].shift(1)) / df['close'].shift(1) ) * 100, 2)

In [15]:
# Sanity check
# (752.17-732.00)/732.00

In [16]:
df['Change'] = pd.cut(df["Price_Change%"], bins = [-np.inf, -5, 5, np.inf], labels=["Decrease", "Neutral", "Increase"])

In [17]:
# Check dataset
df.head()

,open_time,close_time,open,high,low,close,volume,num_trades,MA_10,STD_10,Price_Change,Price_Change%,Change
2020-08-20 02:59:59.999,2020-08-19,2020-08-19 23:59:59.999,50.420,50.420,37.770,42.079,11838.172,4880,NaN,NaN,NaN,NaN,NaN
2020-08-21 02:59:59.999,2020-08-20,2020-08-20 23:59:59.999,42.281,53.880,39.542,46.930,62972.514,18560,NaN,NaN,4.85,11.53,Increase
2020-08-22 02:59:59.999,2020-08-21,2020-08-21 23:59:59.999,46.952,52.100,41.352,41.896,47528.204,19241,NaN,NaN,-5.03,-10.73,Decrease
2020-08-23 02:59:59.999,2020-08-22,2020-08-22 23:59:59.999,42.181,51.000,41.072,47.978,19932.147,8238,NaN,NaN,6.08,14.52,Increase
2020-08-24 02:59:59.999,2020-08-23,2020-08-23 23:59:59.999,48.048,48.057,42.828,43.591,17284.855,6039,NaN,NaN,-4.39,-9.14,Decrease


## Identify Important Dates: Dates where price exceeded 2 stds 

In [18]:
# Create upper and lower bounds
std_times = 3

df['upper_bound'] = df[f"MA_{window}"] + std_times * df[f"STD_{window}"] 
df['lower_bound'] = df[f"MA_{window}"] - std_times * df[f"STD_{window}"] 

In [19]:
# Find cross-points
df['above_bound'] = (df['upper_bound'] < df['high'])
df['below_bound'] = (df['lower_bound'] > df['low'])

In [20]:
## Conclusions

# - There aren't any day crossing both upper and low bound!
# - There are 38 days crossing above bound
# - There are 31 days crossing below bound

print("Total days:", len(df))
df[["above_bound", "below_bound"]].value_counts()

Total days: 405


above_bound  below_bound
False        False          352
True         False           43
False        True             9
True         True             1
dtype: int64

In [21]:
# Create cross_bounds columns [above, below, neutral]

def create_cross_bounds(row):
    if row['above_bound'] :
        return 'above'
    elif row['below_bound'] :
        return 'below'
    else:
        return 'neutral'
    
df["cross_bounds"] = df.apply(lambda row: create_cross_bounds(row), axis=1)

In [22]:
df.head()

,open_time,close_time,open,high,low,close,volume,num_trades,MA_10,STD_10,Price_Change,Price_Change%,Change,upper_bound,lower_bound,above_bound,below_bound,cross_bounds
2020-08-20 02:59:59.999,2020-08-19,2020-08-19 23:59:59.999,50.420,50.420,37.770,42.079,11838.172,4880,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,neutral
2020-08-21 02:59:59.999,2020-08-20,2020-08-20 23:59:59.999,42.281,53.880,39.542,46.930,62972.514,18560,NaN,NaN,4.85,11.53,Increase,NaN,NaN,False,False,neutral
2020-08-22 02:59:59.999,2020-08-21,2020-08-21 23:59:59.999,46.952,52.100,41.352,41.896,47528.204,19241,NaN,NaN,-5.03,-10.73,Decrease,NaN,NaN,False,False,neutral
2020-08-23 02:59:59.999,2020-08-22,2020-08-22 23:59:59.999,42.181,51.000,41.072,47.978,19932.147,8238,NaN,NaN,6.08,14.52,Increase,NaN,NaN,False,False,neutral
2020-08-24 02:59:59.999,2020-08-23,2020-08-23 23:59:59.999,48.048,48.057,42.828,43.591,17284.855,6039,NaN,NaN,-4.39,-9.14,Decrease,NaN,NaN,False,False,neutral


## Get intra day hourly Price for important dates

In [48]:
# Extract start and stop dates
important_dates = df.query("cross_bounds != 'neutral' ")[["open_time","close_time"]]
important_dates = important_dates.reset_index(drop=True)

start_dates = important_dates["open_time"].dt.date.astype(str).values
end_dates = (important_dates["open_time"] + dt.timedelta(days=1)).dt.date.astype(str).values

In [49]:
print("Total interesting dates:", len(start_dates))

Total interesting dates: 53


In [50]:
## Set parameters
root_url = 'https://api.binance.com/api/v1/klines'
symbol = 'NMRUSDT' # 'ETHUSDT'
interval = '1h'
url = root_url + '?symbol=' + symbol + '&interval=' + interval
print(url)

https://api.binance.com/api/v1/klines?symbol=NMRUSDT&interval=1h


In [51]:
def get_intra_date_bars(symbol:str, interval:str, startTime:int, endTime:int):
    """
    Collect data from Binance API
    """
    url = root_url + '?symbol=' + symbol + '&interval=' + interval + '&startTime=' + str(startTime) + '&endTime=' + str(endTime)

    data = json.loads(requests.get(url).text)
    df = pd.DataFrame(data)
    df.columns = ['open_time',
                  'open', 'high', 'low', 'close', 'volume',
                  'close_time', 'qav', 'num_trades',
                  'taker_base_vol', 'taker_quote_vol', 'ignore']
    df.index = [dt.datetime.fromtimestamp(x/1000.0) for x in df.close_time]
    return df

In [52]:
def transform_date_to_unix_timestamp_milliseconds(date:str):
    res = int(dt.datetime.strptime(date, '%Y-%m-%d').strftime("%s")) * 1000
    return res

In [60]:
intra_day_df_full = pd.DataFrame()

for i in range(len(start_dates)):
    
    print("start time:", start_dates[i])
    # print("end time:", end_dates[i])

    startTime = transform_date_to_unix_timestamp_milliseconds(start_dates[i])
    endTime = transform_date_to_unix_timestamp_milliseconds(end_dates[i])
    intra_day_df = get_intra_date_bars(symbol, interval, startTime, endTime)
    
    intra_day_df_full = intra_day_df_full.append(intra_day_df)

start time: 2020-08-28
start time: 2020-10-12
start time: 2020-11-03
start time: 2020-11-17
start time: 2020-11-18
start time: 2020-11-21
start time: 2020-11-22
start time: 2020-11-23
start time: 2020-12-19
start time: 2021-01-05
start time: 2021-01-12
start time: 2021-01-18
start time: 2021-01-19
start time: 2021-02-04
start time: 2021-02-10
start time: 2021-02-20
start time: 2021-02-22
start time: 2021-02-23
start time: 2021-02-26
start time: 2021-02-27
start time: 2021-03-10
start time: 2021-03-11
start time: 2021-03-15
start time: 2021-03-16
start time: 2021-03-17
start time: 2021-03-18
start time: 2021-04-04
start time: 2021-04-05
start time: 2021-04-17
start time: 2021-05-11
start time: 2021-05-16
start time: 2021-05-17
start time: 2021-05-29
start time: 2021-05-30
start time: 2021-06-22
start time: 2021-06-30
start time: 2021-07-13
start time: 2021-07-14
start time: 2021-07-20
start time: 2021-07-25
start time: 2021-08-03
start time: 2021-08-04
start time: 2021-08-07
start time:

In [62]:
# Check intra day 1h price bars
intra_day_df_full.shape

(1325, 12)

In [64]:
# Fix Column types
intra_day_df_full[['open', 'high', 'low', 'close', 'volume']] = intra_day_df_full[['open', 'high', 'low', 'close', 'volume']].apply(pd.to_numeric)

intra_day_df_full['open_time'] = pd.to_datetime(intra_day_df_full['open_time'], unit='ms')
intra_day_df_full['close_time'] = pd.to_datetime(intra_day_df_full['close_time'], unit='ms')

In [66]:
intra_day_df_full.head(30)

,open_time,open,high,low,close,volume,close_time,qav,num_trades,taker_base_vol,taker_quote_vol,ignore
2020-08-28 00:59:59.999,2020-08-27 21:00:00,40.512,43.700,40.297,43.378,1544.286,2020-08-27 21:59:59.999,65430.22539000,540,621.08500000,26385.48562000,0
2020-08-28 01:59:59.999,2020-08-27 22:00:00,43.377,65.124,43.088,63.798,23478.659,2020-08-27 22:59:59.999,1234469.77373200,6520,12053.75900000,637155.23145600,0
2020-08-28 02:59:59.999,2020-08-27 23:00:00,63.798,67.500,50.417,52.437,37126.638,2020-08-27 23:59:59.999,2123157.27710900,8900,16000.88500000,912745.13644700,0
2020-08-28 03:59:59.999,2020-08-28 00:00:00,52.497,56.500,50.000,56.365,17773.320,2020-08-28 00:59:59.999,945433.12155300,3373,8787.41100000,467034.28636400,0
2020-08-28 04:59:59.999,2020-08-28 01:00:00,56.365,57.336,52.396,56.351,6588.705,2020-08-28 01:59:59.999,360080.12132600,2018,3022.05000000,165735.46888200,0
2020-08-28 05:59:59.999,2020-08-28 02:00:00,56.420,56.699,51.274,53.998,3408.496,2020-08-28 02:59:59.999,183206.58018700,1167,1499.31500000,80536.42288300,0
2020-08-28 06:59:59.999,2020-08-28 03:00:00,53.998,53.998,51.506,53.450,1809.462,2020-08-28 03:59:59.999,95650.05238100,405,806.28900000,42703.47180800,0
2020-08-28 07:59:59.999,2020-08-28 04:00:00,53.512,61.500,53.512,61.058,11630.749,2020-08-28 04:59:59.999,678478.30148200,2978,5624.73700000,330006.34035500,0
2020-08-28 08:59:59.999,2020-08-28 05:00:00,60.999,62.842,58.033,58.948,10383.058,2020-08-28 05:59:59.999,628513.83941900,2811,6156.37100000,373368.87262000,0
2020-08-28 09:59:59.999,2020-08-28 06:00:00,58.948,59.006,54.525,54.714,4712.746,2020-08-28 06:59:59.999,265988.07135500,1849,1623.32400000,91999.85591600,0


In [67]:
## Store dataset
intra_day_df_full.to_csv("../data/NMRUSDT_1h_price_data.csv", index=False)

## Plot candlesticks

In [ ]:
import plotly.graph_objects as go
from datetime import datetime

In [ ]:
## Plotly 

# candlestick plot
fig = go.Figure(data=[go.Candlestick(x=df["open_time"],
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'])])

fig.update_layout(
    yaxis_title="USDT",
    title={
        'text': symbol + " Price and cross-points",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

# MA plot
fig.add_trace(
    go.Scatter(
        x = df["open_time"],
        y = df[f"MA_{window}"],
        name = "Moving average",
        mode="lines",
        line=go.scatter.Line(color="black", dash="dash"),
        showlegend=True)
)


# Upper bound
fig.add_trace(
    go.Scatter(
        x= df["open_time"],
        y= df[f"MA_{window}"] + std_times * df[f"STD_{window}"],
        name = "Upper bound",
        mode="lines",
        line=go.scatter.Line(color="lightgreen"),
        showlegend=True)
)

# Lower bound
fig.add_trace(
    go.Scatter(
        x = df["open_time"],
        y = df[f"MA_{window}"] - std_times * df[f"STD_{window}"],
        name = "Lower bound",
        mode="lines",
        line=go.scatter.Line(color="firebrick"),
        showlegend=True)
)


## Plot crossing points

low_value = int(df[['open','high','low','close']].min().min() - 10)

fig.add_trace(go.Scatter(
    x = df.loc[df['above_bound'], "open_time"],
    y = np.repeat(low_value, len(df.loc[df['above_bound'], "open_time"])),
    marker=dict(color="green", size=4),
    mode="markers",
    name = 'above_boundary'
))


fig.add_trace(go.Scatter(
    x = df.loc[df['below_bound'], "open_time"],
    y = np.repeat(low_value, len(df.loc[df['below_bound'], "open_time"])),
    marker=dict(color="crimson", size=4),
    mode="markers",
    name = 'below_boundary'
))


fig.show()

In [ ]:
# Save the plot in the path you want! 
fig.write_html(f"../outcome/{symbol}.html")

## Save final dataset

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
# Store dataset
path = f"../data/{symbol}_{interval}_price_data.csv"
print("Path to save:", path)
df.to_csv(path, index=True)